In [32]:
# Para que las funciones se actualicen si tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
# Funciones propias
from utils_limpieza import * 

In [34]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

In [35]:
# Cargar datos
path = '../data/raw/muestra_diplodatos_ventas_2023.csv'
ventas = pd.read_csv(path)

/tmp/ipykernel_840562/676891302.py:3: DtypeWarning: Columns (10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  ventas = pd.read_csv(path)


## Hacemos la limpieza sugerida al principio del TP2
- [x] Eliminar las variables en `cols_dropear`
- [x] Renombrar a 'Otros' las subcategorías que no tengan `MODELO`
- [ ] Anonimizar los datos que se consideran sensibles

In [36]:
cols_dropear = ['INSCRIPCION', 'CATEGORIA', 'DESCRIPCION_CATEGORIA', 'CATEGORIA (Ajustado)', 'NOMBRE']

ventas_clean = limpiar_basic(ventas, 
                             cols_drop=cols_dropear)

ventas_renamed = renombrar_elementos(ventas_clean, 
                                     columna='SUB-CATEGORIA', 
                                     fill_otros='Otros')

ventas_renamed.groupby(by=['SUB-CATEGORIA'])['MODELO'].sum()

SUB-CATEGORIA
Cigarrillos, tabacos y afines                                               42
Otros                                                                        0
Supermercados, Hipermercados, Kioscos y afines                              42
Venta de Artículos, productos, accesorios, etc de diversos materiales      401
Venta de Combustibles (Incluye tmb gas, carbón, leña, etc)                 393
Venta de Combustibles Ley 23966 (Incluye tmb gas, carbón, leña, etc)       156
Venta de Combustibles para reventa (Incluye tmb gas, carbón, leña, etc)     42
Venta de Productos Alimenticios, Bebidas, Lácteos y afines                 147
Venta de Semillas, Fertilizantes, Granos y afines                           42
Venta en Comisión, Consignación o Intermediación de Productos Varios       126
Name: MODELO, dtype: int64

In [40]:
anonimizar()

función vacía
